In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib.slim import fully_connected as fc
import matplotlib.pyplot as plt 
import readdata
import operator
import os.path
import classifier
from  sklearn.metrics  import  accuracy_score , roc_curve , auc , roc_auc_score
import time
%matplotlib inline

In [ ]:
#Model configurations
hidden_layers = np.asarray([[85, 30, 15], [150, 100,50],[30,20,10],[1000,500,100],
                            [50,30,15],[450,200,100],[35,20,10],[85, 30, 15]])
batch_sizes = np.asarray([100,100,100,100,100,100,100,100])
data_index = 7
dataname = datasets[data_index]
hidden_layer = hidden_layers[data_index]
batch_size = batch_sizes [data_index]

dt = readdata.read_data_sets(dataname)
num_sample = dt.train.num_examples

print ("size of dataset: ",num_sample)
input_dim = dt.train.features.shape[1]

label_dim = dt.train.labels.shape[1]

print ("dimension: ",input_dim)
print  ("number of class: ",label_dim)


data_save = np.asarray([data_index, input_dim,  label_dim])   
data_save = np.reshape(data_save, (-1,3))


if os.path.isfile("Results/datainformation.csv"): #
    auc = np.genfromtxt('Results/datainformation.csv', delimiter=',') 
    auc = np.reshape(auc,(-1,3))
    data_save = np.concatenate((auc, data_save), axis = 0)
    np.savetxt("Results/datainformation.csv", data_save,delimiter = ",",fmt = "%f")
    
else:
    np.savetxt("Results/datainformation.csv", data_save,delimiter = ",",fmt = "%f")

In [ ]:
X_train = dt.train.features
Y_train = dt.train.labels

X_test = dt.test.features
Y_test = dt.test.labels

In [ ]:
class AE(object):

    def __init__(self, learning_rate=1e-3, batch_size=100, hidden_layers = [85, 30,12]):
       
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.hidden_layers = hidden_layers
        self.build()
        self.sess = tf.InteractiveSession()
        self.sess.run(tf.global_variables_initializer())

    # Build the netowrk and the loss functions
    def build(self):
        self.x = tf.placeholder(name='x', dtype=tf.float32, shape=[None, input_dim])
        
        # Encode
        # x -> z_mean, z_sigma -> z
        f1 = fc(self.x, self.hidden_layers[0], scope='ae_enc_fc1', activation_fn=tf.nn.relu)
        #f2 = fc(f1, 60, scope='enc_fc2', activation_fn=tf.nn.tanh)
        f3 = fc(f1, self.hidden_layers[1], scope='ae_enc_fc3', activation_fn=tf.nn.relu)
        #f4 = fc(f3, 20, scope='enc_fc4', activation_fn=tf.nn.relu)
        
        self.z = fc(f3, self.hidden_layers[2], scope='ae_enc_fc5_mu', activation_fn=None)
       
        # Decode
        # z,y -> x_hat
        # g1 = fc(self.Z, 20, scope='dec_fc1', activation_fn=tf.nn.relu)
        g2 = fc(self.z,self.hidden_layers[1], scope='ae_dec_fc2', activation_fn=tf.nn.relu)
        g3 = fc(g2, self.hidden_layers[0], scope='ae_dec_fc3', activation_fn=tf.nn.relu)
        #g4 = fc(g3, 85, scope='dec_fc4', activation_fn=tf.nn.tanh)
       
        self.x_hat = fc(g3, input_dim, scope='ae_dec_fc5', activation_fn=tf.sigmoid)
        #self.x_res = self.x_hat[:,0:input_dim]

        # Loss
        # Reconstruction loss
        recon_loss = tf.reduce_mean(tf.square(self.x - self.x_hat),1) 
        self.recon_loss = tf.reduce_mean(recon_loss)

        self.train_op = tf.train.AdamOptimizer(
            learning_rate=self.learning_rate).minimize(self.recon_loss)
       
        return

    # Execute the forward and the backward pass
    def run_single_step(self, x):
       
        _, recon_loss = self.sess.run(
            [self.train_op,  self.recon_loss],
            feed_dict={self.x: x}
           
        )
      
        return recon_loss
    # x -> x_hat
    def reconstructor(self, x):
        x_hat = self.sess.run(self.x_hat, feed_dict={self.x: x})
        return x_hat

    # z -> x
    def generator(self, z):
        x_hat = self.sess.run(self.x_hat, feed_dict={self.z: z})
        return x_hat
    
    # x -> z
    def transformer(self, x):
        z = self.sess.run(self.z, feed_dict={self.x: x})
        return z

In [ ]:
class MAE(object):

    def __init__(self, learning_rate=1e-3, batch_size=100, hidden_layers = [85, 30,12]):
       
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.hidden_layers = hidden_layers
        self.build()
        self.sess = tf.InteractiveSession()
        self.sess.run(tf.global_variables_initializer())

    # Build the netowrk and the loss functions
    def build(self):
        self.x = tf.placeholder(name='x', dtype=tf.float32, shape=[None, input_dim])
        self.y = tf.placeholder(name='y', dtype=tf.float32, shape=[None, label_dim])
        
        self.Y1 = tf.placeholder(name='Y1', dtype=tf.float32, shape=[None, self.hidden_layers[2]])
        self.Y = tf.arg_max(self.y, 1)
        self.Y_ = tf.to_float(self.Y)
        self.Y1 = tf.expand_dims(self.Y_, 1)
        self.Y1 = self.Y1 + 2.0
       
        # Encode
        # x -> z_mean, z_sigma -> z
        f1 = fc(self.x, self.hidden_layers[0], scope='mae_enc_fc1', activation_fn=tf.nn.relu)
        #f2 = fc(f1, 60, scope='enc_fc2', activation_fn=tf.nn.tanh)
        f3 = fc(f1, self.hidden_layers[1], scope='mae_enc_fc3', activation_fn=tf.nn.relu)
        
        
        self.z = fc(f3, self.hidden_layers[2], scope='mae_enc_fc5_mu', activation_fn=None)
       
        # Decode
        # z,y -> x_hat      
        g2 = fc(self.z,self.hidden_layers[1], scope='mae_dec_fc2', activation_fn=tf.nn.relu)
        g3 = fc(g2, self.hidden_layers[0], scope='mae_dec_fc3', activation_fn=tf.nn.relu)
              
        self.x_hat = fc(g3, input_dim, scope='mae_dec_fc5', activation_fn=tf.sigmoid)
       

        # Loss
        # Reconstruction loss
        # Minimize the cross-entropy loss
        recon_loss = tf.reduce_mean(tf.square(self.x - self.x_hat),1) #(((self.x - y)**2).mean(1)).mean()
        label_loss = tf.reduce_mean(tf.square(self.z - self.Y1),1)
        self.recon_loss = tf.reduce_mean(recon_loss)
        self.label_loss = tf.reduce_mean(label_loss)
        self.total_loss = recon_loss + label_loss

        self.train_op = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.total_loss)  
       
        return

    # Execute the forward and the backward pass
    def run_single_step(self, x, y):
       
        _, recon_loss, label_loss = self.sess.run(
            [self.train_op,  self.recon_loss, self.label_loss],
            feed_dict={self.x: x, self.y: y}
           
        )
      
        return recon_loss, label_loss
    # x -> x_hat
    def reconstructor(self, x):
        x_hat = self.sess.run(self.x_hat, feed_dict={self.x: x})
        return x_hat

    # z -> x
    def generator(self, z):
        x_hat = self.sess.run(self.x_hat, feed_dict={self.z: z})
        return x_hat
    
    # x -> z
    def transformer(self, x):
        z = self.sess.run(self.z, feed_dict={self.x: x})
        return z

In [ ]:
class DMAE(object):

    def __init__(self, learning_rate=1e-3, batch_size=100, hidden_layers = [85, 30,12]):
       
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.hidden_layers = hidden_layers
        self.build()
        self.sess = tf.InteractiveSession()
        self.sess.run(tf.global_variables_initializer())

    # Build the netowrk and the loss functions
    def build(self):
        self.x = tf.placeholder(name='dx', dtype=tf.float32, shape=[None, input_dim])
        self.y = tf.placeholder(name='dy', dtype=tf.float32, shape=[None, label_dim])
        
        self.Y1 = tf.placeholder(name='dY1', dtype=tf.float32, shape=[None, self.hidden_layers[2]])
        self.Y = tf.arg_max(self.y, 1)
        self.Y_ = tf.to_float(self.Y)
        self.Y1 = tf.expand_dims(self.Y_, 1)
        self.Y1 = self.Y1 + 2.0
       
        # Encode
        # x -> z_mean, z_sigma -> z
        X_noise = self.x + self.noise_factor * tf.random_normal(tf.shape(self.x))
        X_noise = tf.clip_by_value(X_noise, 0., 1.)

        # x -> z_mean, z_sigma -> z
        f1 = fc(X_noise, self.hidden_layers[0], scope='dmae_enc_fc1', activation_fn=tf.nn.relu)
        #f2 = fc(f1, 60, scope='enc_fc2', activation_fn=tf.nn.tanh)
        f3 = fc(f1, self.hidden_layers[1], scope='dmae_enc_fc3', activation_fn=tf.nn.relu)
        
        
        self.z = fc(f3, self.hidden_layers[2], scope='dmae_enc_fc5_mu', activation_fn=None)
       
        # Decode
        # z,y -> x_hat      
        g2 = fc(self.z,self.hidden_layers[1], scope='dmae_dec_fc2', activation_fn=tf.nn.relu)
        g3 = fc(g2, self.hidden_layers[0], scope='dmae_dec_fc3', activation_fn=tf.nn.relu)
              
        self.x_hat = fc(g3, input_dim, scope='dmae_dec_fc5', activation_fn=tf.sigmoid)
       

        # Loss
        # Reconstruction loss
        # Minimize the cross-entropy loss
        recon_loss = tf.reduce_mean(tf.square(self.x - self.x_hat),1) 
        label_loss = tf.reduce_mean(tf.square(self.z - self.Y1),1)
        self.recon_loss = tf.reduce_mean(recon_loss)
        self.label_loss = tf.reduce_mean(label_loss)
        self.total_loss = recon_loss + label_loss

        self.train_op = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.total_loss)  
       
        return

    # Execute the forward and the backward pass
    def run_single_step(self, x, y):
       
        _, recon_loss, label_loss = self.sess.run(
            [self.train_op,  self.recon_loss, self.label_loss],
            feed_dict={self.x: x, self.y: y}
           
        )
      
        return recon_loss, label_loss
    # x -> x_hat
    def reconstructor(self, x):
        x_hat = self.sess.run(self.x_hat, feed_dict={self.x: x})
        return x_hat

    # z -> x
    def generator(self, z):
        x_hat = self.sess.run(self.x_hat, feed_dict={self.z: z})
        return x_hat
    
    # x -> z
    def transformer(self, x):
        z = self.sess.run(self.z, feed_dict={self.x: x})
        return z

In [ ]:
class MVAE(object):

    def __init__(self, learning_rate=1e-3, batch_size=100, hidden_layers = [80, 30, 15]):
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.hidden_layers = hidden_layers
        self.build()
        self.sess = tf.InteractiveSession()
        self.sess.run(tf.global_variables_initializer())

    # Build the netowrk and the loss functions
    def build(self):
        self.x = tf.placeholder(name='x', dtype=tf.float32, shape=[None, input_dim])
        self.y = tf.placeholder(name='y', dtype=tf.float32, shape=[None, label_dim])
        
        self.X = tf.placeholder(name='X', dtype=tf.float32, shape=[None, input_dim+label_dim])
        self.Y1 = tf.placeholder(name='Y1', dtype=tf.float32, shape=[None, self.hidden_layers[2]])
        self.X = tf.concat(axis=1, values=[self.x, self.y])
        self.Y = tf.arg_max(self.y, 1)
        self.Y_ = tf.to_float(self.Y)
        self.Y1 = tf.expand_dims(self.Y_, 1)
        self.Y1 = (self.Y1 + 1.0)*2

      # Encode
        # x -> z_mean, z_sigma -> z
        f1 = fc(self.x, self.hidden_layers[0], scope='mvae_enc_fc1', activation_fn=tf.nn.relu)
       
        f3 = fc(f1, self.hidden_layers[1], scope='mvae_enc_fc3', activation_fn=tf.nn.relu)
       
        self.z_mu = fc(f3, self.hidden_layers[2], scope='mvae_enc_fc5_mu', activation_fn=None)
        self.z_log_sigma_sq = fc(f3, self.hidden_layers[2], scope='mvae_enc_fc5_sigma', activation_fn=None)
        eps = tf.random_normal(shape=tf.shape(self.z_log_sigma_sq),
                               mean=0, stddev=1, dtype=tf.float32)
        self.z = self.z_mu + tf.sqrt(tf.exp(self.z_log_sigma_sq)) * eps

        # Decode
        # z,y -> x_hat
        g2 = fc(self.z,self.hidden_layers[1], scope='mvae_dec_fc2', activation_fn=tf.nn.relu)
        g3 = fc(g2, self.hidden_layers[0], scope='mvae_dec_fc3', activation_fn=tf.nn.relu)
           
        self.x_hat = fc(g3, input_dim, scope='mvae_dec_fc5', activation_fn=tf.sigmoid)

        # Loss
        # Reconstruction loss
        # Minimize the mean square error
        recon_loss = tf.reduce_mean(tf.square(self.x - self.x_hat),1) 
        self.recon_loss = tf.reduce_mean(recon_loss)
       
        #MVAE _latent loss
        latent_loss = -0.5*tf.reduce_mean(
            self.z_log_sigma_sq - tf.square(self.z_mu - self.Y1)
            - tf.exp(self.z_log_sigma_sq) + 1, axis=1)
       
        self.latent_loss =1000*tf.reduce_mean(latent_loss)
            
        self.total_loss = recon_loss +latent_loss
        self.train_op = tf.train.AdamOptimizer(
            learning_rate=self.learning_rate).minimize(self.total_loss)
       
        return

    # Execute the forward and the backward pass
    def run_single_step(self, x, y):
        #writer.add_graph(self.sess.graph)
        _, loss, recon_loss, latent_loss = self.sess.run(
            [self.train_op, self.total_loss, self.recon_loss, self.latent_loss],
            feed_dict={self.x: x, self.y: y}
           
        )
      
        return loss, recon_loss, latent_loss
    
    # x -> x_hat
    def reconstructor(self, x, y):
        x_hat = self.sess.run(self.x_hat, feed_dict={self.x: x, self.y: y})
        return x_hat

    # z -> x
    def generator(self, z, y):
        x_hat = self.sess.run(self.x_hat, feed_dict={self.z: z, self.y:y})
        return x_hat
    
    # x -> z
    def transformer(self, x):
        z = self.sess.run(self.z, feed_dict={self.x: x})
        return z
    
    def get_mu_sigma(self, x, y):
        mu, sigma = self.sess.run([self.z_mu,self.z_log_sigma_sq], feed_dict={self.x: x, self.y:y})
        return mu, sigma

In [ ]:
MVAE_loss_ = []
MVAE_recon_loss_=[]
MVAE_latent_loss_ = []
MVAE_auc_svm_ = []
MVAE_auc_dt_ = []
MVAE_auc_rf_ = []

MVAE_t1_ = []
MVAE_t2_ = []
MVAE_t3_ = []
          
def MVAE_trainer(learning_rate=1e-3, batch_size=100, num_epoch=10, hidden_layers = [80, 30, 15]):
    path = "hidden/MVAE/"
    model = MVAE(learning_rate=learning_rate,
                                    batch_size=batch_size, hidden_layers=hidden_layers) 
    for epoch in range(num_epoch):
       
        num_sample = len (X_train)
        for iter in range(num_sample // batch_size):
           
            X_mb, y_mb = dt.train.next_batch(batch_size)
            # Execute the forward and the backward pass and report computed losses
            
            loss, recon_loss, latent_loss= model.run_single_step(X_mb, y_mb)
         
        if epoch % step == 0:
            print('[Epoch {}] Loss: {}, Recon loss: {}, Latent loss: {}'.format(
                epoch, loss, recon_loss, latent_loss))
            #model.writer.add_summary(summary, epoch )
               
            z_train = model.transformer(X_train)
            s = time.time()
            z_test = model.transformer(X_test)
            e = time.time()
            t_tr = (e - s)/ float(len(X_test))
            np.savetxt(path +  "z_train_"+str(epoch)+".csv", z_train, delimiter=",", fmt='%f' )
            np.savetxt(path +  "z_test_"+str(epoch)+".csv", z_train, delimiter=",", fmt='%f' )
            

            #geo, auc, f1, acc = classifier.Gaussian(z_train, Y_train, z_test,Y_test)
            auc_svm, t1 = classifier.svm(z_train, Y_train, z_test,Y_test)
            auc_dt, t2 = classifier.decisiontree(z_train, Y_train, z_test,Y_test)
            auc_rf, t3 = classifier.rf(z_train, Y_train, z_test,Y_test)
           
           
            MVAE_loss_.append(loss)
            MVAE_recon_loss_.append(recon_loss)
            MVAE_latent_loss_.append(latent_loss) 
            MVAE_auc_svm_.append(auc_svm)
            MVAE_auc_dt_.append(auc_dt)
            MVAE_auc_rf_.append(auc_rf)
            MVAE_t1_.append(t1 + t_tr)
            MVAE_t2_.append(t2 + t_tr)
            MVAE_t3_.append(t3 + t_tr)
     
    print('Done MVAE!')
    return model


In [ ]:
AE_recon_loss_=[]
AE_auc_svm_ = []
AE_auc_dt_ = []
AE_auc_rf_ = []

AE_t1_ = []
AE_t2_ = []
AE_t3_ = []

def AE_trainer(learning_rate=1e-3, batch_size=100, num_epoch=10, hidden_layers = [7,4,2]):
    
    model1 = AE(learning_rate=learning_rate, batch_size=batch_size, hidden_layers=hidden_layers)
    for epoch in range(num_epoch):
        
        num_sample = len (X_train)
        for iter in range(num_sample // batch_size):
           
            X_mb, _ = dt.train.next_batch(batch_size)
            # Execute the forward and the backward pass and report computed losses
            recon_loss= model1.run_single_step(X_mb)
        
        if epoch % step == 0:
            print('[Epoch {}] Recon loss: {}'.format(
                epoch, recon_loss))
            #model.writer.add_summary(summary, epoch )
               
            z_train = model1.transformer(X_train)
            s = time.time()
            z_test = model1.transformer(X_test)
            e = time.time()
            t_tr = (e-s)/float(len(X_test))
            #np.savetxt(path +  "z_train_"+str(epoch)+".csv", z_train, delimiter=",", fmt='%f' )
            #np.savetxt(path +  "z_test_"+str(epoch)+".csv", z_train, delimiter=",", fmt='%f' )
            auc_svm, t1 = classifier.svm(z_train, Y_train, z_test,Y_test)
            auc_dt, t2 = classifier.decisiontree(z_train, Y_train, z_test,Y_test)
            auc_rf, t3 = classifier.rf(z_train, Y_train, z_test,Y_test)
             
            AE_recon_loss_.append(recon_loss)
            AE_auc_svm_.append(auc_svm)
            AE_auc_dt_.append(auc_dt)
            AE_auc_rf_.append(auc_rf)
            AE_t1_.append(t1 + t_tr)
            AE_t2_.append(t2 + t_tr)
            AE_t3_.append(t3 + t_tr)
       
    print('Done AE!')
    
   
    return model1


In [ ]:
MAE_recon_loss_=[]
MAE_label_loss_=[]
MAE_auc_svm_ = []
MAE_auc_dt_ = []
MAE_auc_rf_ = []

MAE_t1_ = []
MAE_t2_ = []
MAE_t3_ = []

def MAE_trainer(learning_rate=1e-3, batch_size=100, num_epoch=10, hidden_layers = [7,4,2]):
    path = "hidden/MAE/"
    model1 = MAE(learning_rate=learning_rate, batch_size=batch_size, hidden_layers=hidden_layers)
    for epoch in range(num_epoch):
        
        num_sample = len (X_train)
        for iter in range(num_sample // batch_size):
           
            X_mb, y_mb = dt.train.next_batch(batch_size)
            # Execute the forward and the backward pass and report computed losses
            recon_loss, label_loss= model1.run_single_step(X_mb, y_mb)
        
        if epoch % step == 0:
            print('[Epoch {}] Recon loss: {}'.format(
                epoch, recon_loss))
            #model.writer.add_summary(summary, epoch )
               
            z_train = model1.transformer(X_train)
            s = time.time()
            z_test = model1.transformer(X_test)
            e = time.time()
            t_tr = (e-s)/float(len(X_test))
            np.savetxt(path + "z_train_"+str(epoch)+".csv", z_train, delimiter=",", fmt='%f' )
            np.savetxt(path +"z_test_"+str(epoch)+".csv", z_test, delimiter=",", fmt='%f' )
            auc_svm, t1 = classifier.svm(z_train, Y_train, z_test,Y_test)
            auc_dt, t2 = classifier.decisiontree(z_train, Y_train, z_test,Y_test)
            auc_rf, t3 = classifier.rf(z_train, Y_train, z_test,Y_test)
             
           
           
            MAE_recon_loss_.append(recon_loss)
            MAE_label_loss_.append(label_loss)
            MAE_auc_svm_.append(auc_svm)
            MAE_auc_dt_.append(auc_dt)
            MAE_auc_rf_.append(auc_rf)
            MAE_t1_.append(t1 + t_tr)
            MAE_t2_.append(t2 + t_tr)
            MAE_t3_.append(t3 + t_tr)
       
    print('Done MAE!')
    
   
    return model1


In [ ]:
DMAE_recon_loss_=[]
DMAE_label_loss_=[]
DMAE_auc_svm_ = []
DMAE_auc_dt_ = []
DMAE_auc_rf_ = []

DMAE_t1_ = []
DMAE_t2_ = []
DMAE_t3_ = []

def DMAE_trainer(learning_rate=1e-3, batch_size=100, num_epoch=10, hidden_layers = [7,4,2]):
    path = "hidden/DMAE/"
    model1 = DMAE(learning_rate=learning_rate, batch_size=batch_size, hidden_layers=hidden_layers)
    for epoch in range(num_epoch):   
        num_sample = len (X_train)
        for iter in range(num_sample // batch_size):  
            X_mb, y_mb = dt.train.next_batch(batch_size)
            # Execute the forward and the backward pass and report computed losses
            recon_loss, label_loss= model1.run_single_step(X_mb, y_mb)
        
        if epoch % step == 0:
            print('[Epoch {}] Recon loss: {}'.format(
                epoch, recon_loss))
           
            z_train = model1.transformer(X_train)
            s = time.time()
            z_test = model1.transformer(X_test)
            e = time.time()
            t_tr = (e-s)/float(len(X_test))
            np.savetxt(path + "z_train_"+str(epoch)+".csv", z_train, delimiter=",", fmt='%f' )
            np.savetxt(path +"z_test_"+str(epoch)+".csv", z_test, delimiter=",", fmt='%f' )
            auc_svm, t1 = classifier.svm(z_train, Y_train, z_test,Y_test)
            auc_dt, t2 = classifier.decisiontree(z_train, Y_train, z_test,Y_test)
            auc_rf, t3 = classifier.rf(z_train, Y_train, z_test,Y_test)
               
            DMAE_recon_loss_.append(recon_loss)
            DMAE_label_loss_.append(label_loss)
            DMAE_auc_svm_.append(auc_svm)
            DMAE_auc_dt_.append(auc_dt)
            DMAE_auc_rf_.append(auc_rf)
            DMAE_t1_.append(t1 + t_tr)
            DMAE_t2_.append(t2 + t_tr)
            DMAE_t3_.append(t3 + t_tr)
       
    print('Done DMAE!')
    return model1

In [ ]:
def save_auc_tofile(method, svms, dts, rfs, t1s, t2s, t3s):
    index1, svm = max(enumerate(svms), key=operator.itemgetter(1))
    index2, dt = max(enumerate(dts), key=operator.itemgetter(1))
    index3, rf = max(enumerate(rfs), key=operator.itemgetter(1))
    
    t1 =1000 * sum(t1s)/len(t1s)
    t2 =1000 *  sum(t2s)/len(t2s)
    t3 =1000 *  sum(t3s)/len(t3s)
    

    data_save = np.asarray([data_index, input_dim,  svm, dt, rf, t1, t2, t3])   
    data_save = np.reshape(data_save, (-1,8))


    if os.path.isfile("Results/RF_AUC_DIF/AUC_Hidden_"+method+".csv"): #
        auc = np.genfromtxt("Results/RF_AUC_DIF/AUC_Hidden_"+method+".csv", delimiter=',') 
        auc = np.reshape(auc,(-1,8))
        data_save = np.concatenate((auc, data_save), axis = 0)
        np.savetxt("Results/RF_AUC_DIF/AUC_Hidden_"+method+".csv", data_save,delimiter = ",",fmt = "%f")
    
    else:
        np.savetxt("Results/RF_AUC_DIF/AUC_Hidden_"+method+".csv", data_save,delimiter = ",",fmt = "%f")
    return 1


In [ ]:
# Train the model
num_epoch = 100
step = 20

hidden_layer = hidden_layers[data_index]
block_size = batch_sizes[data_index]
lr = 1e-3
noise_factor = 0.0025

model = MAE_trainer(learning_rate=lr,  batch_size=block_size, num_epoch=num_epoch, hidden_layers = hidden_layer)
#save auc to file:
save_auc_tofile("MAE", MAE_auc_svm_, MAE_auc_dt_, MAE_auc_rf_,MAE_t1_, MAE_t2_, MAE_t3_)

model = DMAE_trainer(learning_rate=lr,  batch_size=block_size, num_epoch=num_epoch, hidden_layers = hidden_layer)
#save auc to file:
save_auc_tofile("DMAE",DMAE_auc_svm_, DMAE_auc_dt_, DMAE_auc_rf_, DMAE_t1_, DMAE_t2_, DMAE_t3_)

model = AE_trainer(learning_rate=lr,  batch_size=block_size, num_epoch=num_epoch, hidden_layers = hidden_layer)
##save auc to file:
save_auc_tofile("AE", AE_auc_svm_, AE_auc_dt_, AE_auc_rf_,AE_t1_, AE_t2_, AE_t3_)

model = MVAE_trainer(learning_rate=lr,  batch_size=block_size, num_epoch=num_epoch, hidden_layers = hidden_layer)
##save auc to file:
save_auc_tofile("MVAE", MVAE_auc_svm_, MVAE_auc_dt_, MVAE_auc_rf_, MVAE_t1_, MVAE_t2_, MVAE_t3_)
